# Generator

Ferramenta que pega no conteudo das noticias (em formato json) e gera posts.
Para isso são executados os seguintes passos:

   1) Obtenção do core do texto:
      -  T5 
      - Allegation sentence

   2) Adição de emoção: 
      - Emojis
      - Hahstags
      - Caps lock
      - interjection

   3) Adição de personalidade



In [1]:
## Importes necessarios

import os

import json
import pandas as pd
import random


In [2]:
'''
verificar se um post (em formato de string) tem um formato válido de um microblog:
     1) 0 < número de carateres ≥ 280 
'''


def verify_length(post):
    content = ""

    for i in post:
        if type(post[i]) == str:
            content = content + " " + post[i] 
        else:
            content = content + " " + ' '.join(post[i]) 
    if len(content)>280:
        return False
    return True

In [3]:
from TextCore import T5 

def allegation_sentence (data):
    claim = "It's " + data['evaluation'] + ' that '+ data['allegation']
    return claim


def post_core (data, core_model):
    
    if core_model == "t5":
        core = T5.T5_summarization_model(' '.join(data['postText']))
    else:
        core = allegation_sentence (data)
    return core

/opt/homebrew/anaconda3/envs/py3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
from emotions import hashtags_generetor
from emotions import emojis
from emotions import interjections
from emotions import caps_lock



def add_emotion(data,core, emoji , hashtag , capslock  , intejections ):
    post = {}
    post['url'] = data['url']
    ## Não adiciona mais conteudo por isso não é necessario verificar tamanho
    if capslock: 
        post['core']  = caps_lock.capslock(core)
    else:
        post['core'] = core
        
    post['emojis'] = []
    if emoji and verify_length(post): 
        post['emojis'].append(emojis.emojis_sellection(data['evaluation']))
   
    post['hashtags'] = []
    if hashtag and verify_length(post):
        hashtag_list = hashtags_generetor.generateHashtags(data['postText'])
        for h in hashtag_list:
            post['hashtags'].append('#'+h)
        
    post['interjections'] = [] 
    if intejections and verify_length(post): 
        post['interjections'].append(interjections.interjections(data['evaluation']))


    if emoji and verify_length(post): 
        #adicionar mais emojis se ainda
        seed = random.randint(0,5)
        while verify_length(post) and seed < 5:
            post['emojis'].append(emojis.emojis_sellection(data['evaluation']))
            seed = seed + 1

    ## Post fica com ordem alaetoria.
   # core_with_emotion = random.shuffle(post) 

    return post

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
def generator (data,core_model='t5', emoji = 1, hashtag = 1, caps_lock =1 , intejections = 1):
  '''
  data -> Conteudo do json file
  '''
  
  ## Obter o core do post
  core = post_core (data, core_model)
  
  ## dar emoção ao core
  core_with_emotion = add_emotion(data, core, emoji , hashtag , caps_lock  , intejections )

  ## dar personalidade
  final_post = core_with_emotion  

  return final_post  

        
        

In [6]:
for file in os.listdir("../../extractors/Snopes/extractions")[:1]:
        with open("../../extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
            print(generator(data, 'allegation_Sentence'))

            

{'url': 'https://www.snopes.com/fact-check/valentines-day-invented-greeting-card-companies/', 'core': 'Its false that VALENTINES DAY was INVENTED by GREETING CARD companies', 'emojis': ['😩', '😡'], 'hashtags': ['#according', '#holiday'], 'interjections': ['oops']}


In [ ]:

##### Read the json files with de news
for file in os.listdir("../../extractors/Snopes/exemplo"):
    with open("../../extractors/Snopes/exemplo/" + file) as f:
        data = json.load(f)
        f.close()
            